## Import Necessary Modules

I'll be using PyTorch for implementing a sample Convolutional Neural Network on a two benchmark data sets for learning:

1. MNIST Dataset
2. Fashion MNIST Dataset (TODO)

In [0]:
import torch.nn as nn
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader # For loading data
from torchvision import transforms, utils
from torchvision import datasets, models, transforms
import pandas as pd
import matplotlib.pyplot as plt

## Load Data

Let's go ahead and load MNIST data to training and validation data.

I'll use PyTorch dataloader class for it.

In [0]:
class MNISTdata(Dataset):
  """
  MNIST Dataset
  ------
  Parameters
    :path: (string) [should be path to a folder]
    :transform: (torchvision.transforms)
    :batch_size: (int) default=8
  """
  def __init__(self, transform, path=None, batch_size=8):
    self.transforms = transform
    self.bs = batch_size
    
    if path is not None:
      self.path = path
      self.train_dir = self.path + "/train"
      self.valid_dir = self.path + "/valid"
      self.data = {
        'train': datasets.ImageFolder(root=self.train_dir, transform=self.transforms['train']),
        'valid': datasets.ImageFolder(root=self.valid_dir, transform=self.transforms['valid']),
      }
    else:
      train_dataset = torchvision.datasets.MNIST(root='data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

      valid_dataset = torchvision.datasets.MNIST(root='data/',
                                                train=False, 
                                                transform=transforms.ToTensor())
      
      self.data = {
          'train': train_dataset,
          'valid': valid_dataset
      }
    
    self.train_loader = DataLoader(self.data['train'], batch_size=self.bs, shuffle=True)
    self.valid_loader = DataLoader(self.data['valid'], batch_size=self.bs, shuffle=True)
    
  def __len__(self):
    return len(self.train_loader)
  
  def __getitem__(self, idx):
    return iter(self.train_loader)[idx]
  
  def get_loaders(self):
    dataloaders = {
        'train': self.train_loader,
        'valid': self.valid_loader
    }
    return dataloaders
  

## Define Transforms for Data Augmentation

In [0]:
image_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.ColorJitter(),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

## Define ConvNet Architecture

In [0]:
class ConvNet(nn.Module):
  """
  Convolutional Neural Network Architecture
  """
  def __init__(self, num_classes=10):
    super(ConvNet, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
      nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Linear(7*7*32, num_classes)
  
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0),-1)
    out = self.fc(out)
    return out

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
num_classes = 10
model = ConvNet(num_classes).to(device)

In [0]:
data = MNISTdata(path=None, transform=image_transforms, batch_size=8)

0it [00:00, ?it/s]

9920512it [00:01, 8768621.50it/s]                            


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 136020.26it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 2238158.86it/s]                            
0it [00:00, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 51659.77it/s]            


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
loaders = data.get_loaders()

## Training Model


In [0]:
# Define hyperparameters
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-2)

In [0]:
def train_and_validate(model, loss_criterion, optimizer, epochs=25):
  """
  Function to train and validate
  
  Parameters
  :param model: Model to train and validate
  :param loss_criterion: Loss Criterion to minimize
  :param optimizer: Optimizer for computing gradients
  :param epochs: Number of epochs (default=25)
  
  Returns
  model: Trained Model with best validation accuracy
  history: (dict object): Having training loss, accuracy and validation loss, accuracy
  """
  train_data = loaders['train']
  valid_data = loaders['valid']
  train_data_size = len(train_data)
  valid_data_size = len(valid_data)
  start = time.time()
  
  # Store best weights to best_weights
  best_weights = copy.deepcopy(model.state_dict())
  
  # Initialize variables and lists
  best_accuracy = 0.0
  history_train_loss = []
  history_val_loss = []
  history_train_acc = []
  history_val_acc = []
  
  for epoch in range(epochs):
    epoch_start = time.time()
    
    print("Epoch: {}/{}".format(epoch+1, epochs))

    # Set to training mode
    model.train()
    
    # Variables for loss and accuracy of each epoch
    train_loss = 0.0
    train_acc = 0
    
    # Iterate through the data
    for _, data in enumerate(train_data):
      inputs, labels = data
      
      inputs = inputs.to(device)
      labels = labels.to(device)
      
      # Clean the existing gradients
      optimizer.zero_grad()
      
      # Calculate outputs on inputs data using the model
      # And find loss and backpropagate
      with torch.set_grad_enabled(True):
        outputs = model(inputs)
        loss = loss_criterion(outputs, labels)

      loss.backward()
      optimizer.step()
      
      # Find loss and accuracy
      
      train_loss += loss.item() * inputs.size(0)
      
      ret, predictions = torch.max(outputs.data, 1)
      correct_counts = predictions.eq(labels.data.view_as(predictions))
      acc = torch.mean(correct_counts.type(torch.FloatTensor))
      train_acc += acc.item() * inputs.size(0)
    
    # Find average training loss and training accuracy
    avg_train_loss = train_loss/train_data_size 
    avg_train_acc = train_acc/float(train_data_size)
    
    epoch_end = time.time()
    
    print("Training: Loss: {:.4f}, Accuracy: {:.4f}, Time: {:.4f}s".format(avg_train_loss, avg_train_acc, epoch_end-epoch_start))
    
    # Start validation 
    
    # Set to evaluation mode
    model.train(False)
    model.eval()
    
    val_start = time.time()
    
    val_loss = 0.0
    val_acc = 0
    
    # Iterate through the data
    for _, data in enumerate(valid_data):
      inputs, labels = data
      
      inputs = inputs.to(device)
      labels = labels.to(device)
      
      # Clean the existing gradients
      optimizer.zero_grad()
      
      # Find outputs from the input data
      outputs = model(inputs)
      
      # Find loss and accuracy
      ret, predictions = torch.max(outputs.data, 1)
      correct_counts = predictions.eq(labels.data.view_as(predictions))
      
      val_loss += loss_criterion(outputs, labels).item() * inputs.size(0)
      acc = torch.mean(correct_counts.type(torch.FloatTensor))
      val_acc += acc.item() * inputs.size(0)
    
    # Calculate average validation loss and accuracy
    avg_val_loss = val_loss/valid_data_size
    avg_val_acc = val_acc/float(valid_data_size)
    
    val_end = time.time()
    
    history_train_loss.append(avg_train_loss)
    history_val_loss.append(avg_val_loss)
    
    history_train_acc.append(avg_train_acc)
    history_val_acc.append(avg_val_acc)
    
    print("Validation: Loss: {:.4f}, Accuracy: {:.4f}, Time: {:.4f}\n".format(avg_val_loss, avg_val_acc, val_end-val_start))
    
    # Save if the model has best accuracy till now
    if(avg_val_acc > best_accuracy):
      best_accuracy = avg_val_acc
      best_weights = copy.deepcopy(model.state_dict())
  
  end = time.time()
  
  print("Total time elapsed: {:.4f}".format(end-start))
  print("Best Val accuracy: {:.4f}".format(best_accuracy))
  
  model.load_state_dict(best_weights)
  
  history = { 'train_loss': history_train_loss, 'val_loss': history_val_loss, 'train_acc': history_train_acc, 'val_acc': history_val_acc}
  return model, history

In [0]:
device

device(type='cuda')

In [0]:
import time
import numpy as np
import copy

save_file_name = 'convnet.pth'
checkpoint_path = 'convnet-checkpoint.pth'

num_epochs = 25
trained_model, history = train_and_validate(model, criterion, optimizer, num_epochs)

Epoch: 1/25
Training: Loss: 18.7002, Accuracy: 0.6595, Time: 23.7408s
Validation: Loss: 18.7072, Accuracy: 0.6888, Time: 1.7544

Epoch: 2/25
Training: Loss: 18.6997, Accuracy: 0.6599, Time: 23.6475s
Validation: Loss: 18.6971, Accuracy: 0.6824, Time: 2.0578

Epoch: 3/25
Training: Loss: 18.7011, Accuracy: 0.6584, Time: 25.1217s
Validation: Loss: 18.6970, Accuracy: 0.6784, Time: 1.7597

Epoch: 4/25
Training: Loss: 18.6989, Accuracy: 0.6603, Time: 23.5196s
Validation: Loss: 18.7030, Accuracy: 0.7040, Time: 1.7597

Epoch: 5/25
Training: Loss: 18.6998, Accuracy: 0.6601, Time: 23.5160s
Validation: Loss: 18.7052, Accuracy: 0.6912, Time: 1.7620

Epoch: 6/25
Training: Loss: 18.7000, Accuracy: 0.6628, Time: 24.8605s
Validation: Loss: 18.6959, Accuracy: 0.6720, Time: 1.7672

Epoch: 7/25
Training: Loss: 18.7004, Accuracy: 0.6616, Time: 23.5647s
Validation: Loss: 18.7237, Accuracy: 0.6776, Time: 1.7580

Epoch: 8/25
Training: Loss: 18.7003, Accuracy: 0.6633, Time: 23.4870s
Validation: Loss: 18.7021, 